In [11]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import math
import numpy as np
import torch
import torch.nn as nn
import torch.utils
import torch.utils.data
# from torch_geometric import nn as tgnn
from preprocessing import sparse_to_tuple
import scipy.sparse as sp
from torch_geometric.utils import *
import torch_scatter
import inspect
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
import tqdm

from models import *

# utility functions

def uniform(size, tensor):
    stdv = 1.0 / math.sqrt(size)
    if tensor is not None:
        tensor.data.uniform_(-stdv, stdv)


def glorot(tensor):
    stdv = math.sqrt(6.0 / (tensor.size(0) + tensor.size(1)))
    if tensor is not None:
        tensor.data.uniform_(-stdv, stdv)


def zeros(tensor):
    if tensor is not None:
        tensor.data.fill_(0)


def ones(tensor):
    if tensor is not None:
        tensor.data.fill_(1)


def reset(nn):
    def _reset(item):
        if hasattr(item, 'reset_parameters'):
            item.reset_parameters()

    if nn is not None:
        if hasattr(nn, 'children') and len(list(nn.children())) > 0:
            for item in nn.children():
                _reset(item)
        else:
            _reset(nn)


def scatter_(name, src, index, dim_size=None):
    r"""Aggregates all values from the :attr:`src` tensor at the indices
    specified in the :attr:`index` tensor along the first dimension.
    If multiple indices reference the same location, their contributions
    are aggregated according to :attr:`name` (either :obj:`"add"`,
    :obj:`"mean"` or :obj:`"max"`).
    Args:
        name (string): The aggregation to use (:obj:`"add"`, :obj:`"mean"`,
            :obj:`"max"`).
        src (Tensor): The source tensor.
        index (LongTensor): The indices of elements to scatter.
        dim_size (int, optional): Automatically create output tensor with size
            :attr:`dim_size` in the first dimension. If set to :attr:`None`, a
            minimal sized output tensor is returned. (default: :obj:`None`)
    :rtype: :class:`Tensor`
    """

    assert name in ['sum', 'mean', 'max']

    op = getattr(torch_scatter, 'scatter_{}'.format(name))
    fill_value = -1e38 if name == 'max' else 0
    print(op(src, index, dim_size=dim_size))

    out = op(src, index, dim = 0, dim_size = dim_size, fill_value = fill_value)
    if isinstance(out, tuple):
        out = out[0]

    if name == 'max':
        out[out == fill_value] = 0

    return out

def tuple_to_array(lot):
    out = np.array(list(lot[0]))
    for i in range(1, len(lot)):
        out = np.vstack((out, np.array(list(lot[i]))))
    
    return out

# masking functions

def mask_edges_det_t(adjs_list):
    adj_train_l, train_edges_l, val_edges_l = [], [], []
    val_edges_false_l, test_edges_l, test_edges_false_l = [], [], []
    edges_list = []

    pbar = tqdm.tqdm(total=len(adjs_list))
    for i in range(0, len(adjs_list)):
        # Function to build test set with 10% positive links
        # NOTE: Splits are randomized and results might slightly deviate from reported numbers in the paper.
        
        adj = adjs_list[i]
        # Remove diagonal elements
        adj = torch.abs(torch.eye(adj.shape[0])*adj - adj)
        # Check that diag is zero:
        assert np.diag(adj.to_dense()).sum() == 0
        
        # Return upper triangle part of array (assumes bidirectional)
        adj_triu = adj.to_dense().triu()
        edges = torch.nonzero(adj_triu)

        # All edges
        edges_all = torch.nonzero(adj.to_dense())
        num_test = int(np.ceil(edges.shape[0]*.10))
        num_val = int(np.ceil(edges.shape[0]*.20))
        
        # Splits all edges to test val train
        all_edge_idx = np.arange(edges.shape[0])
        np.random.shuffle(all_edge_idx)
        val_edge_idx = all_edge_idx[:num_val]
        test_edge_idx = all_edge_idx[num_val:(num_val + num_test)]
        test_edges = edges[test_edge_idx]
        val_edges = edges[val_edge_idx]
        train_edges = np.delete(edges, np.hstack([test_edge_idx, val_edge_idx]), axis=0)
        
        edges_list.append(edges)
        
        def ismember(a, b, tol=5):
            rows_close = np.all(np.round(a - b[:, None], tol) == 0, axis=-1)
            return np.any(rows_close)

        test_edges_false = []
        # Get false test edges 1:1 ratio
        while len(test_edges_false) < len(test_edges):
            idx_i = np.random.randint(0, adj.shape[0])
            idx_j = np.random.randint(0, adj.shape[0])
            if idx_i == idx_j:
                continue
            if ismember([idx_i, idx_j], edges_all):
                continue
            if test_edges_false:
                if ismember([idx_j, idx_i], np.array(test_edges_false)):
                    continue
                if ismember([idx_i, idx_j], np.array(test_edges_false)):
                    continue
            test_edges_false.append([idx_i, idx_j])

        val_edges_false = []
        # Get val edges 1:1 ratio
        while len(val_edges_false) < len(val_edges):
            idx_i = np.random.randint(0, adj.shape[0])
            idx_j = np.random.randint(0, adj.shape[0])
            if idx_i == idx_j:
                continue
            if ismember([idx_i, idx_j], train_edges):
                continue
            if ismember([idx_j, idx_i], train_edges):
                continue
            if ismember([idx_i, idx_j], val_edges):
                continue
            if ismember([idx_j, idx_i], val_edges):
                continue
            if val_edges_false:
                if ismember([idx_j, idx_i], np.array(val_edges_false)):
                    continue
                if ismember([idx_i, idx_j], np.array(val_edges_false)):
                    continue
            val_edges_false.append([idx_i, idx_j])

        assert ~ismember(test_edges_false, edges_all)
        assert ~ismember(val_edges_false, edges_all)
        assert ~ismember(val_edges, train_edges)
        assert ~ismember(test_edges, train_edges)
        assert ~ismember(val_edges, test_edges)

        data = np.ones(train_edges.shape[0])

        # Re-build adj matrix
        adj_train = sp.csr_matrix((data, (train_edges[:, 0], train_edges[:, 1])), shape=adj.shape)
        adj_train = adj_train + adj_train.T

        # Adj_train is full matrix
        adj_train_l.append(adj_train)
        train_edges_l.append(train_edges)
        val_edges_l.append(val_edges)
        val_edges_false_l.append(val_edges_false)
        test_edges_l.append(test_edges)
        test_edges_false_l.append(test_edges_false)
        pbar.update(1)
    pbar.close()

    # NOTE: these edge lists only contain single direction of edge!
    return adj_train_l, train_edges_l, val_edges_l, val_edges_false_l, test_edges_l, test_edges_false_l

def mask_edges_prd_t(adjs_list):
    pos_edges_l , false_edges_l = [], []
    edges_list = []

    pbar = tqdm.tqdm(total=len(adjs_list))
    for i in range(0, len(adjs_list)):
        # Function to build test set with 10% positive links
        # NOTE: Splits are randomized and results might slightly deviate from reported numbers in the paper.
        
        adj = adjs_list[i]
        # Remove diagonal elements
        adj = torch.abs(torch.eye(adj.shape[0])*adj - adj)
        # Check that diag is zero:
        assert np.diag(adj.todense()).sum() == 0
        
        # Return upper triangle part of array (assumes bidirectional)
        adj_triu = sp.triu(adj)
        adj_tuple = sparse_to_tuple(adj_triu)
        edges = adj_tuple[0]

        # All edges
        edges_all = sparse_to_tuple(adj)[0]
        num_false = int(edges.shape[0])
        
        # Positive edges
        pos_edges_l.append(edges)
        
        def ismember(a, b, tol=5):
            rows_close = np.all(np.round(a - b[:, None], tol) == 0, axis=-1)
            return np.any(rows_close)
        
        # Retrieve negative edges
        edges_false = []
        while len(edges_false) < num_false:
            idx_i = np.random.randint(0, adj.shape[0])
            idx_j = np.random.randint(0, adj.shape[0])
            if idx_i == idx_j:
                continue
            if ismember([idx_i, idx_j], edges_all):
                continue
            if edges_false:
                if ismember([idx_j, idx_i], np.array(edges_false)):
                    continue
                if ismember([idx_i, idx_j], np.array(edges_false)):
                    continue
            edges_false.append([idx_i, idx_j])

        assert ~ismember(edges_false, edges_all)
        
        false_edges_l.append(edges_false)

        pbar.update(1)
    pbar.close()
    # NOTE: these edge lists only contain single direction of edge!
    return pos_edges_l, false_edges_l

def mask_edges_prd_new_t(adjs_list):
    pos_edges_l , false_edges_l = [], []
    edges_list = []
    
    # Function to build test set with 10% positive links
    # NOTE: Splits are randomized and results might slightly deviate from reported numbers in the paper.

    adj = adjs_list[0]
    # Remove diagonal elements
    adj = torch.abs(torch.eye(adj.shape[0])*adj - adj)
    # Check that diag is zero:
    assert np.diag(adj.todense()).sum() == 0

    # Return upper triangle part of array (assumes bidirectional)
    adj_triu = sp.triu(adj)
    adj_tuple = sparse_to_tuple(adj_triu)
    edges = adj_tuple[0]

    # All edges
    edges_all = sparse_to_tuple(adj)[0]
    num_false = int(edges.shape[0])

    # Positive edges
    pos_edges_l.append(edges)

    def ismember(a, b, tol=5):
        rows_close = np.all(np.round(a - b[:, None], tol) == 0, axis=-1)
        return np.any(rows_close)

    # Generate negative edges
    edges_false = []
    while len(edges_false) < num_false:
        idx_i = np.random.randint(0, adj.shape[0])
        idx_j = np.random.randint(0, adj.shape[0])
        if idx_i == idx_j:
            continue
        if ismember([idx_i, idx_j], edges_all):
            continue
        if edges_false:
            if ismember([idx_j, idx_i], np.array(edges_false)):
                continue
            if ismember([idx_i, idx_j], np.array(edges_false)):
                continue
        edges_false.append([idx_i, idx_j])

    assert ~ismember(edges_false, edges_all)    
    false_edges_l.append(np.asarray(edges_false))
    
    pbar = tqdm.tqdm(total=len(adjs_list))
    for i in range(1, len(adjs_list)):
        # Get newly generated edges 
        edges_pos = np.transpose(np.asarray(np.nonzero((adjs_list[i] - adjs_list[i-1])>0)))
        num_false = int(edges_pos.shape[0])
        
        adj = adjs_list[i]
        # Remove diagonal elements
        adj = torch.abs(torch.eye(adj.shape[0])*adj - adj)
        # Check that diag is zero:
        assert np.diag(adj.todense()).sum() == 0
        
        # Get upper triangle part of array (assumes bidirectional)
        adj_triu = sp.triu(adj)
        adj_tuple = sparse_to_tuple(adj_triu)
        edges = adj_tuple[0]
        edges_all = sparse_to_tuple(adj)[0]
        
        # Get negative edges for each newly generated edge
        edges_false = []
        while len(edges_false) < num_false:
            idx_i = np.random.randint(0, adj.shape[0])
            idx_j = np.random.randint(0, adj.shape[0])
            if idx_i == idx_j:
                continue
            if ismember([idx_i, idx_j], edges_all):
                continue
            if edges_false:
                if ismember([idx_j, idx_i], np.array(edges_false)):
                    continue
                if ismember([idx_i, idx_j], np.array(edges_false)):
                    continue
            edges_false.append([idx_i, idx_j])
        
        assert ~ismember(edges_false, edges_all)
        
        false_edges_l.append(np.asarray(edges_false))
        pos_edges_l.append(edges_pos)
    
        pbar.update(1)
    pbar.close()
    # NOTE: these edge lists only contain single direction of edge!
    return pos_edges_l, false_edges_l

# evaluation function

def get_roc_scores(edges_pos, edges_neg, adj_orig_dense_list, embs):
    def sigmoid(x):
        return 1 / (1 + np.exp(-x))
    
    auc_scores = []
    ap_scores = []
    
    for i in range(len(edges_pos)):
        # Predict on test set of edges
        emb = embs[i].detach().numpy()
        adj_rec = np.dot(emb, emb.T)
        adj_orig_t = adj_orig_dense_list[i].todense()
        preds = []
        pos = []
        for e in edges_pos[i]:
            preds.append(sigmoid(adj_rec[e[0], e[1]]))
            pos.append(adj_orig_t[e[0], e[1]])
            
        preds_neg = []
        neg = []
        for e in edges_neg[i]:
            preds_neg.append(sigmoid(adj_rec[e[0], e[1]]))
            neg.append(adj_orig_t[e[0], e[1]])
        
        preds_all = np.hstack([preds, preds_neg])
        labels_all = np.hstack([np.ones(len(preds)), np.zeros(len(preds_neg))])
        auc_scores.append(roc_auc_score(labels_all, preds_all))
        ap_scores.append(average_precision_score(labels_all, preds_all))

    return auc_scores, ap_scores

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import math
import numpy as np
import torch
import torch.nn as nn
import torch.utils
import torch.utils.data
from torch.autograd import Variable
# from torch_geometric import nn as tgnn
from preprocessing import sparse_to_tuple, get_starboard_data
import scipy.sparse as sp
from torch.nn.parameter import Parameter
import torch.nn.functional as F
import time
from torch_scatter import scatter_mean, scatter_max, scatter_add
from torch_geometric.utils import *
from torch_geometric.nn import MessagePassing
import torch_scatter
import inspect
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
import copy
import pickle
import tqdm

from models import *

seed = 123
np.random.seed(seed)

cuda_num = 5
device = torch.device('mps' if torch.backends.mps.is_available() else f'cuda:{cuda_num}' if torch.cuda.is_available() else 'cpu')

In [2]:
with open('/data/mala711/GNNthesis/data/Starboard/adj_time_list.pickle', 'rb') as handle:
    adj_time_list = pickle.load(handle)

interval = -460
interval = -50
adj_time_list = adj_time_list[interval:]

adj_time_list_t = torch.stack(adj_time_list).to(device)

In [4]:
# Function to build test set with 10% positive links
# NOTE: Splits are randomized and results might slightly deviate from reported numbers in the paper.

adj = adj_time_list_t[0].to(device)
# Remove diagonal elements
adj = torch.abs(torch.eye(adj.shape[0]).to(device)*adj - adj)
adj_all = adj + adj.T
# Check that diag is zero:
assert torch.diag(adj.to_dense()).sum() == 0

In [26]:
# Return upper triangle part of array (assumes bidirectional)
edges = torch.nonzero(adj.to_dense())

# All edges
edges_all = torch.nonzero(adj_all.to_dense())
num_test = int(np.ceil(edges.shape[0]*.10))
num_val = int(np.ceil(edges.shape[0]*.20))

# Splits all edges to test val train
all_edge_idx = torch.randperm(edges.shape[0])
val_edge_idx = all_edge_idx[:num_val]
test_edge_idx = all_edge_idx[num_val:(num_val + num_test)]
test_edges = edges[test_edge_idx]
val_edges = edges[val_edge_idx]
train_edges = edges[~(torch.hstack([test_edge_idx, val_edge_idx]))]

In [77]:
def ismember(a, b):
    rows_close = torch.all(torch.round(a - b[:, None]) == 0, dim = 1)
    return torch.any(rows_close)

test_edges_false = torch.empty(0).to(device)
# Get false test edges 1:1 ratio
while len(test_edges_false) < len(test_edges):
    idx_i = np.random.randint(0, adj.shape[0])
    idx_j = np.random.randint(0, adj.shape[0])
    coord1 = torch.Tensor([idx_i, idx_j]).to(device)
    coord2 = torch.Tensor([idx_j, idx_i]).to(device)
    if idx_i == idx_j:
        continue
    if ismember(coord1, edges_all):
        continue
    if test_edges_false.shape[0] > 0:
        if ismember(coord1, test_edges_false):
            continue
        if ismember(coord2, test_edges_false):
            continue
    test_edges_false = torch.cat((test_edges_false, coord1), 0)
test_edges_false = test_edges_false.reshape(-1, 2)

val_edges_false = torch.empty(0).to(device)
# Get val edges 1:1 ratio
while len(val_edges_false) < len(val_edges):
    idx_i = np.random.randint(0, adj.shape[0])
    idx_j = np.random.randint(0, adj.shape[0])
    coord1 = torch.Tensor([idx_i, idx_j]).to(device)
    coord2 = torch.Tensor([idx_j, idx_i]).to(device)
    if idx_i == idx_j:
        continue
    if ismember(coord1, train_edges):
        continue
    if ismember(coord2, train_edges):
        continue
    if ismember(coord1, val_edges):
        continue
    if ismember(coord2, val_edges):
        continue
    if val_edges_false.shape[0] > 0:
        if ismember(coord1, val_edges_false):
            continue
        if ismember(coord2, val_edges_false):
            continue
    val_edges_false = torch.cat((val_edges_false, coord1), 0)
val_edges_false = val_edges_false.reshape(-1, 2)

In [79]:
train_edges

tensor([[ 1573,  1400],
        [ 2224,  1932],
        [ 4221,  1012],
        [ 8130,   576],
        [  375,  2021],
        [ 3246,  2227],
        [ 9475,  6955],
        [ 1581,   600],
        [ 8408, 10802],
        [ 1868, 10751],
        [ 9515,   978],
        [ 1510,  1365],
        [ 4205,  3966],
        [ 8093,  1636],
        [ 1163,  1302],
        [ 3494,  5897],
        [ 3722,  3723],
        [ 5392,  6349],
        [ 5324,  1437],
        [ 6224,  1095],
        [   58,    59],
        [ 8520,  2288],
        [ 4837,  1880],
        [ 2862,  2150],
        [ 6387,  3656],
        [ 5185,   951],
        [ 3166,  1854],
        [ 1720,  5439],
        [ 1599,  7378],
        [ 2774,  1863],
        [ 8057,  5229],
        [ 6068,  1773],
        [ 2611,  1184],
        [ 1423,  6706],
        [ 6200,  1166],
        [ 7829,  1336],
        [ 3429,  3212],
        [ 4724,  1910],
        [ 9568,  1134],
        [ 5764,  4045],
        [ 2154,  3909],
        [ 4590, 

In [80]:
assert ~ismember(test_edges_false, edges_all)
assert ~ismember(val_edges_false, edges_all)
assert ~ismember(val_edges, train_edges)
assert ~ismember(test_edges, train_edges)
assert ~ismember(val_edges, test_edges)

In [88]:
cur = adj_time_list_t[1].to(device).to_dense()
prev = adj_time_list_t[0].to(device).to_dense()
edges_pos = torch.nonzero((cur - prev) > 0)
num_false = int(edges_pos.shape[0])

In [ ]:
edges_pos = np.transpose(np.asarray(np.nonzero((cur - prev)>0)))
num_false = int(edges_pos.shape[0])

In [ ]:
coords = adj_new.coalesce().indices()
values = adj_new.coalesce().values()
coordinates = coords[:, torch.where(values > 0)[0]]

coordinates

In [ ]:
# Retrieves train_edges adjacency list
outs = mask_edges_det_t(adj_time_list)
train_edges_l = outs[1]

In [ ]:


# masking edges



# Get negative and positive edges of adjacency list and returns a bidirectional adjacecny list
pos_edges_l, false_edges_l = mask_edges_prd(adj_time_list)

# Get negative and positive edges that are newly generated (i has no edge but i+1 has one)
pos_edges_l_n, false_edges_l_n = mask_edges_prd_new(adj_time_list)


# creating edge list

edge_idx_list = []

for i in range(len(train_edges_l)):
    edge_idx_list.append(torch.tensor(np.transpose(train_edges_l[i]), dtype=torch.long))

In [ ]:
# hyperparameters

h_dim = 32
z_dim = 16
n_layers =  1
clip = 10
learning_rate = 1e-2
seq_len = len(train_edges_l)
num_nodes = adj_time_list[seq_len-1].shape[0]
x_dim = num_nodes
eps = 1e-10
conv_type='GCN'
epochs = 50



# creating input tensors per sequence length

x_in_list = []

x_temp = torch.eye(num_nodes)
x_temp = x_temp.expand(seq_len, num_nodes, num_nodes)
x_in = Variable(x_temp).to(device)


# building model

model = VGRNN(x_dim, h_dim, z_dim, n_layers, eps, conv=conv_type, bias=True).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


# training

seq_start = 0
seq_end = seq_len - 3
tst_after = 0

for k in range(epochs):
    optimizer.zero_grad()
    start_time = time.time()
    kld_loss, nll_loss, _, _, hidden_st = model(x_in[seq_start:seq_end]
                                                , edge_idx_list[seq_start:seq_end]
                                                , adj_time_list[seq_start:seq_end])
    loss = kld_loss + nll_loss
    loss.backward()
    optimizer.step()
    
    nn.utils.clip_grad_norm(model.parameters(), clip)
    
    if k>tst_after:
        _, _, enc_means, pri_means, _ = model(x_in[seq_end:seq_len]
                                              , edge_idx_list[seq_end:seq_len]
                                              , adj_time_list[seq_end:seq_len]
                                              , hidden_st)
        
        auc_scores_prd, ap_scores_prd = get_roc_scores(pos_edges_l[seq_end:seq_len]
                                                        , false_edges_l[seq_end:seq_len]
                                                        , adj_time_list[seq_end:seq_len]
                                                        , pri_means)
        
        auc_scores_prd_new, ap_scores_prd_new = get_roc_scores(pos_edges_l_n[seq_end:seq_len]
                                                                , false_edges_l_n[seq_end:seq_len]
                                                                , adj_time_list[seq_end:seq_len]
                                                                , pri_means)
        
    
    print('epoch: ', k)
    print('kld_loss =', kld_loss.mean().item())
    print('nll_loss =', nll_loss.mean().item())
    print('loss =', loss.mean().item())
    if k>tst_after:
        print('----------------------------------')
        print('Link Prediction')
        print('link_prd_auc_mean', np.mean(np.array(auc_scores_prd)))
        print('link_prd_ap_mean', np.mean(np.array(ap_scores_prd)))
        print('----------------------------------')
        print('New Link Prediction')
        print('new_link_prd_auc_mean', np.mean(np.array(auc_scores_prd_new)))
        print('new_link_prd_ap_mean', np.mean(np.array(ap_scores_prd_new)))
        print('----------------------------------')
    print('----------------------------------')
